In [1]:
import pandas as pd
import numpy as np
import tempfile
import tensorflow as tf

# linus 下面这个包是不是没用？
# from six.moves import urllib

import pandas as pd

In [3]:
data1 = pd.read_pickle('../data/dataV0623.pkl') #这里是数据


In [15]:
# 这里loc和iloc有一个小区别：是否包含9999
train = data1.loc[:9999] 
train

,sex,age,currentAddress,currentPayment,postJobName,recommendReason,workLonger,eduBackground,cvkeywords,major,...,benefit,label,EqualAddress,EqualJob,Cvs,Jds,EqualWord,Cvsnum,Jdsnum,FitEdu
0,MALE,0.517857,3,0.351719,"[-0.48266388745405814, -0.05315832288594922, -...","[-1.8090081402864413, 0.3290084377321029, -0.4...",0.068182,本科,"[1.167181915595595, 0.7402713343413235, -0.002...",自动化,...,"[2.28408191352732, 3.1790636116043856, -1.1752...",1,False,0.000000,"[-2.6323798639569413, -8.362326446566954, 6.59...","[-4.006641716357684, -0.30150849242412653, 2.5...",0.015319,0.098091,0.264808,True
1,MALE,0.517857,3,0.351719,"[-0.48266388745405814, -0.05315832288594922, -...","[-1.8090081402864413, 0.3290084377321029, -0.4...",0.068182,本科,"[1.167181915595595, 0.7402713343413235, -0.002...",自动化,...,"[-0.8078333176741346, 0.10183298300898035, 0.1...",0,False,0.000000,"[-2.6323798639569413, -8.362326446566954, 6.59...","[-1.6276091508113422, 0.20449902033825904, 0.2...",0.032340,0.098091,0.243902,True
2,MALE,0.625000,2,0.615905,"[2.189816649559665, 0.8339484106039369, -0.379...","[-1.7793489433135383, 0.31867071284006326, -0....",0.272727,本科,"[-0.27545664632821076, 0.20010831589142028, 0....",其他,...,"[-0.8078333176741346, 0.10183298300898035, 0.1...",1,False,0.214286,"[-8.988107651674683, -0.267331026154027, -0.26...","[4.2935560277093066, -2.0177516063965832, 1.71...",0.011631,0.046605,0.261324,True
3,MALE,0.625000,2,0.615905,"[2.189816649559665, 0.8339484106039369, -0.379...","[-1.7793489433135383, 0.31867071284006326, -0....",0.272727,本科,"[-0.27545664632821076, 0.20010831589142028, 0....",其他,...,"[1.8133310443946342, -0.1626848538637713, 0.39...",0,False,0.000000,"[-8.988107651674683, -0.267331026154027, -0.26...","[-2.3890939903339015, -0.38455091868784547, 1....",0.005390,0.046605,0.303136,True
4,MALE,0.535714,4,0.683741,"[1.0948469288848115, -0.19021169859424483, 0.2...","[-0.4294405618602598, 0.42602468109693004, -0....",0.181818,大专,"[-1.08261333362462, 0.794929018900652, -0.3579...",经济,...,"[-0.028005999236623502, -0.4466591456950586, -...",1,False,0.000000,"[2.976442904804161, 0.6720510893970607, -11.90...","[-0.8522593081768158, -2.183819430180423, -1.4...",0.006809,0.094541,0.052265,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,MALE,0.732143,1,0.911020,"[-0.517073595301277, -0.08738882373231693, -0....","[8.932784905341462, 1.4482439901692261, -1.803...",0.386364,硕士,"[-0.721097793491382, 1.8760672714047575, -0.08...",管理,...,"[-0.8078333176741346, 0.10183298300898035, 0.1...",0,False,0.000000,"[33.491870110934336, -1.91172254347215, -11.99...","[-1.3635810857710364, 0.26230930772368516, -0....",0.043404,0.250777,0.160279,True
9996,FEMALE,0.589286,1,0.692063,"[2.5700115327779782, 0.24630990029448063, 0.07...","[2.235171782409259, 0.0012722958390820002, 0.5...",0.204545,本科,"[0.29349750947087916, 3.5233780914934254, 0.76...",语言,...,"[-0.028005999236623502, -0.4466591456950586, -...",1,False,0.000000,"[29.19256595109075, -16.88702557545301, -10.28...","[-0.8522593081768158, -2.183819430180423, -1.4...",0.017872,0.242787,0.052265,True
9997,FEMALE,0.589286,1,0.692063,"[2.5700115327779782, 0.24630990029448063, 0.07...","[2.235171782409259, 0.0012722958390820002, 0.5...",0.204545,本科,"[0.29349750947087916, 3.5233780914934254, 0.76...",语言,...,"[-0.8078333176741346, 0.10183298300898035, 0.1...",0,False,0.000000,"[29.19256595109075, -16.88702557545301, -10.28...","[-1.3598502464650581, -1.035682429326033, 1.46...",0.020426,0.242787,0.195122,True
9998,MALE,0.625000,0,0.502823,"[-0.5079663002457584, -0.08051085888972376, -0...","[2.3054762927682444, -1.2632639937085444, -0.0...",0.272727,本科,"[-0.2550317730773661, -0.13078890461622703, 0....",管理,...,"[-0.8078333176741346, 0.10183298300898035, 0.1...",1,True,0.000000,"[10.533326983489594, 12.674381778599276, 4.937...","[-1.1813318221060505, 1.209225779852382, -1.40...",0.026383,0.176653,0.219512,True


In [33]:
# 对col1列进行特征离散化

# num_oov_buckets会新增2列，防止有新词分配给他们, 加上词表的4列onehot之后一共有6列
one_categorical_feature = tf.feature_column.categorical_column_with_vocabulary_list(
            'col1', vocabulary_list=['a', 'x', 'ca', '我'], num_oov_buckets=2)
# onehot的索引顺序就是['a', 'x', 'ca', ''我']，所以只要指定了vocabulary_list的顺序，结果都是一样的
one_categorical_feature_show = tf.feature_column.indicator_column(one_categorical_feature)  

features = {'col1': [['a'], ['我'], ['a'], ['ca']]}
# 这种输入也可以
#features = {'col1': ['a', '我', 'a', 'ca']}

net = tf.feature_column.input_layer(features, one_categorical_feature_show)
with tf.Session() as sess:
    sess.run(tf.tables_initializer())
    print(net.eval())

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [25]:
pd.DataFrame(features)

,col1
0,[a]
1,[我]
2,[a]
3,[ca]
